In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [84]:
def compute_stars(comment):
    pt_batch = tokenizer(comment, padding=True, truncation=True, max_length=512, return_tensors="pt")
    pt_outputs = model(**pt_batch)
    tensor = F.softmax(pt_outputs.logits, dim=-1)
    
    def compute_avg_stars(vals):
        return (1 * vals[0]) + (2 * vals[1]) + (3 * vals[2]) + (4 * vals[3]) + (5 * vals[4])

    return list(map(compute_avg_stars, tensor.tolist()))
    
    #return { "1": vals[0], "2": vals[1], "3": vals[2], "4": vals[3], "5": vals[4], "avg": avg_stars }
    

In [1]:
import pandas as pd
import numpy as np

reviews_path = '../data/amazon-reviews.json'
products_path = '../data/amazon-products.json'
stars_path = '../data/computed_stars.json'

reviews_df = pd.read_json(reviews_path)
products_df = pd.read_json(products_path)
stars_df = pd.read_json(stars_path, typ="series")

In [131]:
products_df

,_id,categories,storeId,storeName,price,href,origin,protocol,host,pathname,productId,productTitle,productGlobalReview,numberOfRatings,__v
0,60a64efdc6b6ff187482412d,"[Beauty & Personal Care, Makeup, Makeup Remove...",3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Neutrogena,None,https://www.amazon.com/Neutrogena-Oil-Free-Eye...,https://www.amazon.com,https:,www.amazon.com,/Neutrogena-Oil-Free-Eye-Makeup-Remover/dp/B00...,B000NWAOHE,"Neutrogena Oil-Free Liquid Eye Makeup Remover,...",4.7,19'015,0
1,60a64f40c6b6ff187482425a,"[Beauty & Personal Care, Skin Care, Face, Crea...",0CA2A9B7-7695-43AF-98D9-FB55761E0796,CeraVe,None,https://www.amazon.com/CeraVe-Facial-Moisturiz...,https://www.amazon.com,https:,www.amazon.com,/CeraVe-Facial-Moisturizing-Lotion-AM/dp/B00F9...,B00F97FHAW,CeraVe AM Facial Moisturizing Lotion SPF 30 | ...,4.5,33'334,0
2,60a64f42c6b6ff1874824387,"[Beauty & Personal Care, Skin Care, Face, Clea...",0CA2A9B7-7695-43AF-98D9-FB55761E0796,CeraVe,None,https://www.amazon.com/CeraVe-Hydrating-Facial...,https://www.amazon.com,https:,www.amazon.com,/CeraVe-Hydrating-Facial-Cleanser-Fragrance/dp...,B01MSSDEPK,CeraVe Hydrating Facial Cleanser | Moisturizin...,4.7,51'173,0
3,60a64f81c6b6ff18748243a8,"[Beauty & Personal Care, Skin Care, Face, Clea...",3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Neutrogena,None,https://www.amazon.com/Neutrogena-Cleansing-Hy...,https://www.amazon.com,https:,www.amazon.com,/Neutrogena-Cleansing-Hydrating-Oil-Free-Sensi...,B07767Z9YZ,Neutrogena Hydro Boost Gentle Cleansing and Hy...,4.6,100,0
4,60a65012c6b6ff18748244d5,"[Beauty & Personal Care, Skin Care, Body, Clea...",9B9CC7AD-3D51-4456-9A0C-0614D1D92F13,Dove,None,https://www.amazon.com/Dove-Body-Wash-Pump-Moi...,https://www.amazon.com,https:,www.amazon.com,/Dove-Body-Wash-Pump-Moisture/dp/B00MEDOY2G/re...,B00MEDOY2G,Dove Body Wash with Pump with Skin Natural Nou...,4.8,34'808,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,60a6b53f6dccd831e8d98bdb,"[Home & Kitchen, Kitchen & Dining, Dining & En...",8780CBF9-8B22-4381-8D37-0AAC20F190F2,American Metalcraft,None,https://www.amazon.com/American-Metalcraft-MEL...,https://www.amazon.com,https:,www.amazon.com,/American-Metalcraft-MELSQ117-Endurance-228-Ou...,B002XH17BE,American Metalcraft MELSQ117 Endurance Melamin...,4.6,278,0
1856,60a6b5726dccd831e8d98d08,"[Office Products, Office & School Supplies, Bi...",947C6949-CF8E-4BD3-914A-B411DD3E4433,Amazon Basics,None,https://www.amazon.com/AmazonBasics-D-Ring-Bin...,https://www.amazon.com,https:,www.amazon.com,/AmazonBasics-D-Ring-Binder-White-4-Pack/dp/B0...,B077XCJ6S1,"Amazon Basics Binder - 2 Inch D-Ring, White, 4...",4.7,4'622,0
1857,60a6b5776dccd831e8d98e1b,"[Patio, Lawn & Garden, Gardening & Lawn Care, ...",1D2A0DDE-57D0-4030-8B1F-5C33E3A4A94F,Fiskars,None,https://www.amazon.com/Fiskars-29-Inch-Machete...,https://www.amazon.com,https:,www.amazon.com,/Fiskars-29-Inch-Machete-Axe/dp/B019EXCS2Q/ref...,B019EXCS2Q,Fiskars 29 Inch Machete Axe,4.6,814,0
1858,60a6b57c6dccd831e8d98f43,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",4E822E20-FC23-45E9-BCD6-3275BBFBAF5E,Colorations,None,https://www.amazon.com/Colorations-SWTALL-Simp...,https://www.amazon.com,https:,www.amazon.com,/Colorations-SWTALL-Simply-Washable-Tempera/dp...,B005E9K0DY,"Colorations Simply Washable Tempera Paint, 19C...",4.7,4'434,0


In [132]:
import psycopg2
from tqdm import tqdm
import math

conn = psycopg2.connect(**{
    "host": "localhost",
    "database": "hochet",
    "user": "hochet",
    "password": "hochet"
})

def clean(s):
    return s.replace("'", "").replace('"', '')

cursor = conn.cursor()
cols = ','.join(["productid", "categories", "storeid", "storename", "href", "origin", "protocol", "host", "pathname", "producttitle", "productglobalreview", "numberofratings"])
already_inserted = []

for item in tqdm(products_df.T.to_dict().values()):
    if item['productId'] not in already_inserted:
        values = (
            item['productId'],
            item['categories'],
            item['storeId'],
            item['storeName'],
            item['href'],
            item['origin'],
            item['protocol'],
            item['host'],
            item['pathname'],
            item['productTitle'],
            item['productGlobalReview'],
            item['numberOfRatings'].replace("'", '') if item['numberOfRatings'] is not None else 0
        )

        query = "INSERT INTO public.products ({}) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(cols)
        cursor.execute(query, values)
        already_inserted.append(item['productId'])

try:
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    conn.rollback()
    cursor.close()
cursor.close()



100%|████████████████████████████████████████████████████████████████████████████| 1860/1860 [00:00<00:00, 2268.04it/s]


In [2]:
def compute_next_cols(row):
    stars = compute_stars([row.title, row.content, row.title + " " + row.content])
    return {
        "title_computed_stars": stars[0],
        "content_computed_stars": stars[1],
        "title_concat_content_computed_stars": stars[2],
    }

In [3]:
computed_stars_reviews_df = reviews_df.apply(compute_next_cols, result_type="expand", axis=1)

NameError: name 'compute_stars' is not defined

In [4]:
reviews2_df = pd.concat([reviews_df, pd.DataFrame(stars_df, columns=['computed_stars'])], axis=1)
reviews2_df

,_id,productId,storeId,author,authorImg,authorLink,stars,title,fromAndDate,content,foundHelpful,__v,computed_stars
0,60a64efdc6b6ff1874824029,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Steven,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great product as long as you follow the direct...,"Reviewed in the United States on May 1, 2016",I was initially so disappointed when I tried t...,15,0,3.955002
1,60a64efdc6b6ff187482402e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Alix,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,what does oil free actually mean?,"Reviewed in the United States on October 17, 2017",I don't know if oil free means your face will ...,3,0,2.085644
2,60a64efdc6b6ff187482403e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Razzle Dazzle,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,Doesn't remove all the makeup and greasy!,"Reviewed in the United States on June 19, 2019",I was using an oil based makeup remover and it...,None,0,2.028057
3,60a64efdc6b6ff1874824052,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Amazon Customer,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Two thumbs up!,"Reviewed in the United States on December 15, ...",Impressive product. Right now I’m using Volum...,,0,4.845288
4,60a64efdc6b6ff1874824053,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Aundrea,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,4.0,"It's a good makeup remover, but does leave an ...","Reviewed in the United States on June 7, 2016","This has been working out great for me, but I ...",4,0,3.291641
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161570,60a6b57d6dccd831e8d9906b,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,joAnne,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Safety first,"Reviewed in the United States on April 23, 2018","Easy to install, make sure you order one long ...",None,0,4.278356
161571,60a6b57d6dccd831e8d9906c,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Pools,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great,"Reviewed in the United States on July 19, 2019",Best one out there the cable doesn't drag or g...,None,0,4.674066
161572,60a6b57d6dccd831e8d9906d,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Mark Gilbertville,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Good value,"Reviewed in the United States on July 24, 2019",REMEMBER 4ft is completely extended,None,0,4.285427
161573,60a6b57d6dccd831e8d9906e,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Jonathan Couture,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Heavy clips and rugged,"Reviewed in the United States on December 17, ...",Always a huge fan of these things. Simple easy...,None,0,4.725660


In [5]:
from datetime import datetime
import time

def format_location_date(txt):
    date = datetime.strptime(" ".join(txt.split(" ")[-3:]), "%B %d, %Y")
    location = " ".join(txt.split(" ")[2:-4])
    return pd.Series({"date": date, "location": location })

In [6]:
reviews2_location_date_serie = reviews2_df.apply(lambda row: format_location_date(row.fromAndDate), axis=1)
reviews2_location_date = pd.DataFrame(reviews2_location_date_serie)

In [7]:
reviews3_df = pd.concat([reviews2_df, reviews2_location_date], axis=1)
reviews3_df

,_id,productId,storeId,author,authorImg,authorLink,stars,title,fromAndDate,content,foundHelpful,__v,computed_stars,date,location
0,60a64efdc6b6ff1874824029,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Steven,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great product as long as you follow the direct...,"Reviewed in the United States on May 1, 2016",I was initially so disappointed when I tried t...,15,0,3.955002,2016-05-01,the United States
1,60a64efdc6b6ff187482402e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Alix,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,what does oil free actually mean?,"Reviewed in the United States on October 17, 2017",I don't know if oil free means your face will ...,3,0,2.085644,2017-10-17,the United States
2,60a64efdc6b6ff187482403e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Razzle Dazzle,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,Doesn't remove all the makeup and greasy!,"Reviewed in the United States on June 19, 2019",I was using an oil based makeup remover and it...,None,0,2.028057,2019-06-19,the United States
3,60a64efdc6b6ff1874824052,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Amazon Customer,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Two thumbs up!,"Reviewed in the United States on December 15, ...",Impressive product. Right now I’m using Volum...,,0,4.845288,2017-12-15,the United States
4,60a64efdc6b6ff1874824053,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Aundrea,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,4.0,"It's a good makeup remover, but does leave an ...","Reviewed in the United States on June 7, 2016","This has been working out great for me, but I ...",4,0,3.291641,2016-06-07,the United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161570,60a6b57d6dccd831e8d9906b,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,joAnne,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Safety first,"Reviewed in the United States on April 23, 2018","Easy to install, make sure you order one long ...",None,0,4.278356,2018-04-23,the United States
161571,60a6b57d6dccd831e8d9906c,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Pools,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great,"Reviewed in the United States on July 19, 2019",Best one out there the cable doesn't drag or g...,None,0,4.674066,2019-07-19,the United States
161572,60a6b57d6dccd831e8d9906d,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Mark Gilbertville,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Good value,"Reviewed in the United States on July 24, 2019",REMEMBER 4ft is completely extended,None,0,4.285427,2019-07-24,the United States
161573,60a6b57d6dccd831e8d9906e,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Jonathan Couture,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Heavy clips and rugged,"Reviewed in the United States on December 17, ...",Always a huge fan of these things. Simple easy...,None,0,4.725660,2018-12-17,the United States


In [21]:
def authorId(link):
    return pd.Series({"authorId": None if link is None else link.split('.')[-1].split('/')[0] })

reviews3_authorId = reviews3_df.apply(lambda row: authorId(row.authorLink), axis=1)
reviews3_authorId_df = pd.DataFrame(reviews3_authorId)

reviews4_df = pd.concat([reviews3_df, reviews3_authorId_df], axis=1)
reviews4_df

,_id,productId,storeId,author,authorImg,authorLink,stars,title,fromAndDate,content,foundHelpful,__v,computed_stars,date,location,authorId
0,60a64efdc6b6ff1874824029,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Steven,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great product as long as you follow the direct...,"Reviewed in the United States on May 1, 2016",I was initially so disappointed when I tried t...,15,0,3.955002,2016-05-01,the United States,AGPNBTY3BSECMCV5NI52L64NDTPA
1,60a64efdc6b6ff187482402e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Alix,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,what does oil free actually mean?,"Reviewed in the United States on October 17, 2017",I don't know if oil free means your face will ...,3,0,2.085644,2017-10-17,the United States,AEQEJHZVVP726OWARX2B66MSKQGQ
2,60a64efdc6b6ff187482403e,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Razzle Dazzle,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,2.0,Doesn't remove all the makeup and greasy!,"Reviewed in the United States on June 19, 2019",I was using an oil based makeup remover and it...,None,0,2.028057,2019-06-19,the United States,AEWBUQA56UXZBIP6C5YZH3237XLA
3,60a64efdc6b6ff1874824052,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Amazon Customer,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Two thumbs up!,"Reviewed in the United States on December 15, ...",Impressive product. Right now I’m using Volum...,,0,4.845288,2017-12-15,the United States,AGCEVLASYODXFXBIEIOJJSOLFTDQ
4,60a64efdc6b6ff1874824053,B000NWAOHE,3BC8CF14-3F35-41A4-9BF3-705A7FB2B6B6,Aundrea,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,4.0,"It's a good makeup remover, but does leave an ...","Reviewed in the United States on June 7, 2016","This has been working out great for me, but I ...",4,0,3.291641,2016-06-07,the United States,AFFCTK6Y27DONGPEBDIGH6YXH2TQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161570,60a6b57d6dccd831e8d9906b,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,joAnne,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Safety first,"Reviewed in the United States on April 23, 2018","Easy to install, make sure you order one long ...",None,0,4.278356,2018-04-23,the United States,AG3FTTXLX6GUQ5LAPJP3QJ5D5JEA
161571,60a6b57d6dccd831e8d9906c,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Pools,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Great,"Reviewed in the United States on July 19, 2019",Best one out there the cable doesn't drag or g...,None,0,4.674066,2019-07-19,the United States,AE74Y7KNDXTATL7HZPLUAPSE3XFQ
161572,60a6b57d6dccd831e8d9906d,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Mark Gilbertville,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Good value,"Reviewed in the United States on July 24, 2019",REMEMBER 4ft is completely extended,None,0,4.285427,2019-07-24,the United States,AGRFJFC5YJ44KFJSDFMFZWALH7UA
161573,60a6b57d6dccd831e8d9906e,B00718W5GQ,AE9F00D1-C11E-4A9B-AFEB-B4EA056EF019,Jonathan Couture,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/gp/profile/amzn1.accoun...,5.0,Heavy clips and rugged,"Reviewed in the United States on December 17, ...",Always a huge fan of these things. Simple easy...,None,0,4.725660,2018-12-17,the United States,AHNWXGCEWYP2Q44UUQ2QUQXS7MCQ


In [49]:
import psycopg2
from tqdm import tqdm
import math
import json

conn = psycopg2.connect(**{
    "host": "localhost",
    "database": "hochet",
    "user": "hochet",
    "password": "hochet"
})

def clean(s):
    return s.replace("'", "").replace('"', '')

def execute_many(conn, df):
    cursor = conn.cursor()
    tuples = [tuple(x) for x in df.to_numpy()]
    i = 1
    
    for item in tqdm(tuples):
        productid = item[0]
        storeid = item[1]
        author = item[2].replace("'", "").replace('"', "")
        authorImg = item[3]
        authorLink = item[4]
        stars = item[5]
        title = item[6].replace("'", "").replace('"', "") if item[6] is not None else None
        content = item[8].replace("'", "").replace('"', "") if item[8] is not None else None
        computed_stars = item[10]
        date = item[11]
        location = item[12]
        authorId = item[13]
        if authorId is not None:
            query = "INSERT INTO public.updated_reviews (id, productid, storeid, author, stars, computed_stars, date, location, author_img, author_link, title, content, author_id) VALUES ({}, '{}', '{}', '{}', {}, {}, '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(i, productid, storeid, author, stars, computed_stars, date, location, authorImg, authorLink, title, content, authorId)
            cursor.execute(query)
            i = i + 1
    
    try:
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        conn.rollback()
        cursor.close()
    cursor.close()
    

execute_many(conn, reviews4_df.drop(columns=['_id', '__v']))

100%|████████████████████████████████████████████████████████████████████████| 161575/161575 [01:08<00:00, 2372.27it/s]
